<a href="https://colab.research.google.com/github/17002235uhi/17002235uhi_DataAnalytics/blob/main/3%20-%20DNN_Train_and_Test_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training and Testing DNN Model

In [ ]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/17002235uhi/17002235uhi_DataAnalytics/main/dnndata.csv', index_col=0)

In [ ]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

   Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  Sun  Thu  Tue  Wed  \
1    0    0    0    0    1    0    0    0    0    0  ...    0    0    0    0   
2    0    0    0    0    1    0    0    0    0    0  ...    0    0    1    0   
3    0    0    0    0    1    0    0    0    0    0  ...    0    0    0    1   
4    0    0    0    0    1    0    0    0    0    0  ...    0    1    0    0   
5    0    0    0    0    1    0    0    0    0    0  ...    0    0    0    0   
6    0    0    0    0    1    0    0    0    0    0  ...    0    0    0    0   

   year  temp  wdsp  visib  dewp  NUM_CRIMES  
1  2017  34.0   5.2    7.2  27.9    0.188679  
2  2017  38.2   5.9    2.4  36.9    0.518868  
3  2017  18.2  14.9    9.7   9.0    0.185535  
4  2017   9.3  11.5    9.9  -1.5    0.182390  
5  2017   4.4  10.2   10.0  -6.4    0.150943  
6  2017   6.5   8.0   10.0  -4.8    0.204403  

[6 rows x 25 columns]


In [ ]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:24]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  Sat  Sun  Thu  \
827     0    0    0    0    0    0    0    0    1    0  ...    1    0    0   
378     0    0    0    1    0    0    0    0    0    0  ...    0    0    1   
763     0    0    0    0    0    0    0    1    0    0  ...    0    0    1   
1310    0    0    0    0    0    0    0    0    0    0  ...    0    1    0   
793     1    0    0    0    0    0    0    0    0    0  ...    0    1    0   
59      0    0    0    0    0    0    0    1    0    0  ...    0    0    0   

      Tue  Wed  year  temp  wdsp  visib  dewp  
827     0    0  2019  73.6   9.4   10.0  62.7  
378     0    0  2018  27.8  11.6   10.0  13.9  
763     0    0  2019  39.8   9.0    8.0  34.8  
1310    0    0  2020  61.3   9.0   10.0  51.3  
793     0    0  2019  58.3   5.9   10.0  25.3  
59      0    1  2017  47.1   9.4    5.6  43.8  

[6 rows x 24 columns]


In [ ]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,...,Sun,Thu,Tue,Wed,year,temp,wdsp,visib,dewp,NUM_CRIMES
827,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,2019,73.6,9.4,10.0,62.7,1.000000
378,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,2018,27.8,11.6,10.0,13.9,0.635135
763,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,2019,39.8,9.0,8.0,34.8,0.425926
1310,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,2020,61.3,9.0,10.0,51.3,0.752044
793,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,2019,58.3,5.9,10.0,25.3,0.290123


In [ ]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,24]

# print out the first 6 rows of the targets data.
print(targets[:6])

827     1.000000
378     0.635135
763     0.425926
1310    0.752044
793     0.290123
59      0.591195
Name: NUM_CRIMES, dtype: float64


In [ ]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_CRIMES = 1.0

In [ ]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_CRIMES']) - trainsize

# Define the number of input values (predictors)
nppredictors = 24
# Define the number of output values (targets)
noutputs = 1

In [ ]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_CRIMES, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_CRIMES

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa2ce2ce450>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow

In [ ]:
#"Apr","Aug","Dec","Feb","Jan","Jul","Jun","Mar","May","Nov","Oct","Sep","Fri","Mon","Sat","Sun","Thu","Tue","Wed","year","temp","wdsp","NUM_CRIMES"
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [1,0,0],
         'Dec' : [0,1,0],
         'Feb' : [0,0,1],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [1,1,1],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'year' : [2017,2018,2019],
         'temp' : [61.8, 31.2, 40.0],
         'wdsp' : [5.0, 3.0, 8.0],
         'visib' : [10.0, 9.8, 7.6],
         'dewp' : [27.9, 36.9, 9.0],
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa2cdf1b2d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model/mo